Angie Joya - 2322609

In [1]:
import psutil
print(f'Numero de CPUs lógicas: {psutil.cpu_count()}')

Numero de CPUs lógicas: 12


**Versión Secuencial**

In [2]:
%%writefile main.c

#include <stdio.h>
#include <stdlib.h>
#include <string.h>

// Funciones placeholder para la carga y guardado de imágenes
void cargarImagen(int *imagen, int width, int height);
void guardarImagen(int *imagen, int width, int height);

// Función para aplicar un filtro simple
void aplicarFiltro(int *imagen, int *imagenProcesada, int width, int height);

// Función para calcular la suma de los píxeles (como una estadística)
int calcularSumaPixeles(int *imagen, int width, int height);

char *filename;

int main(int argc, char* argv[]) {
    int width = atoi(argv[2]);
    int height = atoi(argv[3]); 
    int *imagen = (int *)malloc(width * height * sizeof(int));
    int *imagenProcesada = (int *)malloc(width * height * sizeof(int));

    if (argc != 4) {
      fprintf(stderr, "Dar un nombre de archivo de entrada");
      exit(1);
    }

    filename = argv[1];
    // Cargar la imagen (no paralelizable)
    cargarImagen(imagen, width, height);

    // Aplicar filtro (paralelizable)
    aplicarFiltro(imagen, imagenProcesada, width, height);

    // Calcular suma de píxeles (parte paralelizable)
    int sumaPixeles = calcularSumaPixeles(imagenProcesada, width, height);

    printf("Suma de píxeles: %d\n", sumaPixeles);

    // Guardar la imagen (no paralelizable)
    guardarImagen(imagenProcesada, width, height);

    free(imagen);
    free(imagenProcesada);
    return 0;
}

// Carga una imagen desde un archivo binario
void cargarImagen(int *imagen, int width, int height) {
    FILE *archivo = fopen(filename, "rb");
    if (archivo == NULL) {
        perror("Error al abrir el archivo para cargar la imagen");
        return;
    }

    size_t elementosLeidos = fread(imagen, sizeof(int), width * height, archivo);
    if (elementosLeidos != width * height) {
        perror("Error al leer la imagen desde el archivo");
    }

    fclose(archivo);
}

// Guarda una imagen en un archivo binario
void guardarImagen(int *imagen, int width, int height) {
    char *output_filename;

    output_filename = (char*)malloc(sizeof(char)*(strlen(filename) + 4));
    sprintf(output_filename,"%s.new",filename);
    FILE *archivo = fopen(output_filename, "wb");
    if (archivo == NULL) {
        perror("Error al abrir el archivo para guardar la imagen");
        return;
    }

    size_t elementosEscritos = fwrite(imagen, sizeof(int), width * height, archivo);
    if (elementosEscritos != width * height) {
        perror("Error al escribir la imagen en el archivo");
    }

    fclose(archivo);
}


void aplicarFiltro(int *imagen, int *imagenProcesada, int width, int height) {
    int Gx[3][3] = {{-1, 0, 1}, {-2, 0, 2}, {-1, 0, 1}};
    int Gy[3][3] = {{-1, -2, -1}, {0, 0, 0}, {1, 2, 1}};

    for (int y = 1; y < height - 1; y++) {
        for (int x = 1; x < width - 1; x++) {
            int sumX = 0;
            int sumY = 0;

            // Aplicar máscaras de Sobel (Gx y Gy)
            for (int ky = -1; ky <= 1; ky++) {
                for (int kx = -1; kx <= 1; kx++) {
                    sumX += imagen[(y + ky) * width + (x + kx)] * Gx[ky + 1][kx + 1];
                    sumY += imagen[(y + ky) * width + (x + kx)] * Gy[ky + 1][kx + 1];
                    //sumX += imagen[(y + ky), x + kx] * Gx[ky + 1, kx + 1]
                    //sumY += imagen[(y + ky), x + kx] * Gy[ky + 1, kx + 1]
                }
            }

            // Calcular magnitud del gradiente
            int magnitude = abs(sumX) + abs(sumY);
            imagenProcesada[y * width + x] = (magnitude > 255) ? 255 : magnitude;  // Normalizar a 8 bits
        }
    }
}


int calcularSumaPixeles(int *imagen, int width, int height) {
    int suma = 0;
    for (int i = 0; i < width * height; i++) {
        suma += imagen[i];
    }
    return suma;
}

Overwriting main.c


In [3]:
!time make
!time make
!time make
!time make
!time make

gcc -o main main.c
time ./all.sh
Suma de píxeles: 18903643
Suma de píxeles: 19538173
Suma de píxeles: 508153298
Suma de píxeles: 239735997
Suma de píxeles: 346448124
Suma de píxeles: 156622739
Suma de píxeles: 190237953
Suma de píxeles: 310060410
Suma de píxeles: 32052138
Suma de píxeles: 84940088
Suma de píxeles: 109861856
Suma de píxeles: 467284619
Suma de píxeles: 529723150
Suma de píxeles: 512155570
Suma de píxeles: 196109827
Suma de píxeles: 34497298
Suma de píxeles: 130193667
Suma de píxeles: 438836200
28.40user 33.03system 0:24.82elapsed 247%CPU (0avgtext+0avgdata 158596maxresident)k
856inputs+1643624outputs (33major+682395minor)pagefaults 0swaps
cat all.sh >all 
chmod a+x all

real	0m24,874s
user	0m28,449s
sys	0m33,046s
gcc -o main main.c
time ./all.sh
Suma de píxeles: 18903643
Suma de píxeles: 19538173
Suma de píxeles: 508153298
Suma de píxeles: 239735997
Suma de píxeles: 346448124
Suma de píxeles: 156622739
Suma de píxeles: 190237953
Suma de píxeles: 310060410
Suma de píxeles

**Versión Paralela - Hilos = Nucleos**

In [11]:
%%writefile main.c

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <omp.h>

// Funciones placeholder para la carga y guardado de imágenes
void cargarImagen(int *imagen, int width, int height);
void guardarImagen(int *imagen, int width, int height);

// Función para aplicar un filtro simple
void aplicarFiltro(int *imagen, int *imagenProcesada, int width, int height);

// Función para calcular la suma de los píxeles (como una estadística)
int calcularSumaPixeles(int *imagen, int width, int height);

char *filename;

int main(int argc, char* argv[]) {
    int width = atoi(argv[2]);
    int height = atoi(argv[3]); 
    int *imagen = (int *)malloc(width * height * sizeof(int));
    int *imagenProcesada = (int *)malloc(width * height * sizeof(int));

    if (argc != 4) {
      fprintf(stderr, "Dar un nombre de archivo de entrada");
      exit(1);
    }

    filename = argv[1];
    // Cargar la imagen (no paralelizable)
    cargarImagen(imagen, width, height);

    
    aplicarFiltro(imagen, imagenProcesada, width, height);
    int sumaPixeles = calcularSumaPixeles(imagenProcesada, width, height);
    printf("Suma de píxeles: %d\n", sumaPixeles);
    

    // Guardar la imagen (no paralelizable)
    guardarImagen(imagenProcesada, width, height);

    free(imagen);
    free(imagenProcesada);
    return 0;
}

// Carga una imagen desde un archivo binario
void cargarImagen(int *imagen, int width, int height) {
    FILE *archivo = fopen(filename, "rb");
    if (archivo == NULL) {
        perror("Error al abrir el archivo para cargar la imagen");
        return;
    }

    size_t elementosLeidos = fread(imagen, sizeof(int), width * height, archivo);
    if (elementosLeidos != width * height) {
        perror("Error al leer la imagen desde el archivo");
    }

    fclose(archivo);
}

// Guarda una imagen en un archivo binario
void guardarImagen(int *imagen, int width, int height) {
    char *output_filename;

    output_filename = (char*)malloc(sizeof(char)*(strlen(filename) + 4));
    sprintf(output_filename,"%s.new",filename);
    FILE *archivo = fopen(output_filename, "wb");
    if (archivo == NULL) {
        perror("Error al abrir el archivo para guardar la imagen");
        return;
    }

    size_t elementosEscritos = fwrite(imagen, sizeof(int), width * height, archivo);
    if (elementosEscritos != width * height) {
        perror("Error al escribir la imagen en el archivo");
    }

    fclose(archivo);
}


void aplicarFiltro(int *imagen, int *imagenProcesada, int width, int height) {
    int Gx[3][3] = {{-1, 0, 1}, {-2, 0, 2}, {-1, 0, 1}};
    int Gy[3][3] = {{-1, -2, -1}, {0, 0, 0}, {1, 2, 1}};
     
    #pragma omp parallel for
    
    for (int y = 1; y < height - 1; y++) {
        for (int x = 1; x < width - 1; x++) {
            int sumX = 0;
            int sumY = 0;

            // Aplicar máscaras de Sobel (Gx y Gy)
            for (int ky = -1; ky <= 1; ky++) {
                for (int kx = -1; kx <= 1; kx++) {
                    sumX += imagen[(y + ky) * width + (x + kx)] * Gx[ky + 1][kx + 1];
                    sumY += imagen[(y + ky) * width + (x + kx)] * Gy[ky + 1][kx + 1];
                    //sumX += imagen[(y + ky), x + kx] * Gx[ky + 1, kx + 1]
                    //sumY += imagen[(y + ky), x + kx] * Gy[ky + 1, kx + 1]
                }
            }

            // Calcular magnitud del gradiente
            int magnitude = abs(sumX) + abs(sumY);
            imagenProcesada[y * width + x] = (magnitude > 255) ? 255 : magnitude;  // Normalizar a 8 bits
        }
    }
    
}


int calcularSumaPixeles(int *imagen, int width, int height) {
    int suma = 0;
    #pragma omp parallel for reduction(+:suma)
    
    for (int i = 0; i < width * height; i++) {
        suma += imagen[i];
    }
    
    return suma;
}

Overwriting main.c


In [13]:
!time make
!time make
!time make
!time make
!time make

gcc -fopenmp -o main main.c
time ./all.sh
Suma de píxeles: 18903643
Suma de píxeles: 19538173
Suma de píxeles: 508153298
Suma de píxeles: 239735997
Suma de píxeles: 346448124
Suma de píxeles: 156622739
Suma de píxeles: 190237953
Suma de píxeles: 310060410
Suma de píxeles: 32052138
Suma de píxeles: 84940088
Suma de píxeles: 109861856
Suma de píxeles: 467284619
Suma de píxeles: 529723150
Suma de píxeles: 512155570
Suma de píxeles: 196109827
Suma de píxeles: 34497298
Suma de píxeles: 130193667
Suma de píxeles: 438836200
39.45user 35.25system 0:21.39elapsed 349%CPU (0avgtext+0avgdata 158456maxresident)k
0inputs+1643632outputs (29major+683567minor)pagefaults 0swaps
cat all.sh >all 
chmod a+x all

real	0m21,467s
user	0m39,501s
sys	0m35,282s
gcc -fopenmp -o main main.c
time ./all.sh
Suma de píxeles: 18903643
Suma de píxeles: 19538173
Suma de píxeles: 508153298
Suma de píxeles: 239735997
Suma de píxeles: 346448124
Suma de píxeles: 156622739
Suma de píxeles: 190237953
Suma de píxeles: 310060410

**Versión Paralela - Hilos = 2 x Nucleos**

In [18]:
%%writefile main.c

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <omp.h>

// Funciones placeholder para la carga y guardado de imágenes
void cargarImagen(int *imagen, int width, int height);
void guardarImagen(int *imagen, int width, int height);

// Función para aplicar un filtro simple
void aplicarFiltro(int *imagen, int *imagenProcesada, int width, int height);

// Función para calcular la suma de los píxeles (como una estadística)
int calcularSumaPixeles(int *imagen, int width, int height);

char *filename;

int main(int argc, char* argv[]) {
    int width = atoi(argv[2]);
    int height = atoi(argv[3]); 
    int *imagen = (int *)malloc(width * height * sizeof(int));
    int *imagenProcesada = (int *)malloc(width * height * sizeof(int));
    
    if (argc != 4) {
      fprintf(stderr, "Dar un nombre de archivo de entrada");
      exit(1);
    }

    filename = argv[1];
    // Cargar la imagen (no paralelizable)
    cargarImagen(imagen, width, height);

    
    aplicarFiltro(imagen, imagenProcesada, width, height);
    int sumaPixeles = calcularSumaPixeles(imagenProcesada, width, height);
    printf("Suma de píxeles: %d\n", sumaPixeles);
    

    // Guardar la imagen (no paralelizable)
    guardarImagen(imagenProcesada, width, height);

    free(imagen);
    free(imagenProcesada);
    return 0;
}

// Carga una imagen desde un archivo binario
void cargarImagen(int *imagen, int width, int height) {
    FILE *archivo = fopen(filename, "rb");
    if (archivo == NULL) {
        perror("Error al abrir el archivo para cargar la imagen");
        return;
    }

    size_t elementosLeidos = fread(imagen, sizeof(int), width * height, archivo);
    if (elementosLeidos != width * height) {
        perror("Error al leer la imagen desde el archivo");
    }

    fclose(archivo);
}

// Guarda una imagen en un archivo binario
void guardarImagen(int *imagen, int width, int height) {
    char *output_filename;

    output_filename = (char*)malloc(sizeof(char)*(strlen(filename) + 4));
    sprintf(output_filename,"%s.new",filename);
    FILE *archivo = fopen(output_filename, "wb");
    if (archivo == NULL) {
        perror("Error al abrir el archivo para guardar la imagen");
        return;
    }

    size_t elementosEscritos = fwrite(imagen, sizeof(int), width * height, archivo);
    if (elementosEscritos != width * height) {
        perror("Error al escribir la imagen en el archivo");
    }

    fclose(archivo);
}


void aplicarFiltro(int *imagen, int *imagenProcesada, int width, int height) {
    int Gx[3][3] = {{-1, 0, 1}, {-2, 0, 2}, {-1, 0, 1}};
    int Gy[3][3] = {{-1, -2, -1}, {0, 0, 0}, {1, 2, 1}};
    
    int cantHilos = 2*omp_get_num_procs();
     
    #pragma omp parallel for num_threads(cantHilos)
    
    for (int y = 1; y < height - 1; y++) {
        for (int x = 1; x < width - 1; x++) {
            int sumX = 0;
            int sumY = 0;

            // Aplicar máscaras de Sobel (Gx y Gy)
            for (int ky = -1; ky <= 1; ky++) {
                for (int kx = -1; kx <= 1; kx++) {
                    sumX += imagen[(y + ky) * width + (x + kx)] * Gx[ky + 1][kx + 1];
                    sumY += imagen[(y + ky) * width + (x + kx)] * Gy[ky + 1][kx + 1];
                    //sumX += imagen[(y + ky), x + kx] * Gx[ky + 1, kx + 1]
                    //sumY += imagen[(y + ky), x + kx] * Gy[ky + 1, kx + 1]
                }
            }

            // Calcular magnitud del gradiente
            int magnitude = abs(sumX) + abs(sumY);
            imagenProcesada[y * width + x] = (magnitude > 255) ? 255 : magnitude;  // Normalizar a 8 bits
        }
    }
    
}


int calcularSumaPixeles(int *imagen, int width, int height) {
    int suma = 0;
    int cantHilos = 2*omp_get_num_procs();
    #pragma omp parallel for reduction(+:suma) num_threads(cantHilos)
    
    for (int i = 0; i < width * height; i++) {
        suma += imagen[i];
    }
    
    return suma;
}

Overwriting main.c


In [20]:
!time make
!time make
!time make
!time make
!time make

gcc -fopenmp -o main main.c
time ./all.sh
Suma de píxeles: 18903643
Suma de píxeles: 19538173
Suma de píxeles: 508153298
Suma de píxeles: 239735997
Suma de píxeles: 346448124
Suma de píxeles: 156622739
Suma de píxeles: 190237953
Suma de píxeles: 310060410
Suma de píxeles: 32052138
Suma de píxeles: 84940088
Suma de píxeles: 109861856
Suma de píxeles: 467284619
Suma de píxeles: 529723150
Suma de píxeles: 512155570
Suma de píxeles: 196109827
Suma de píxeles: 34497298
Suma de píxeles: 130193667
Suma de píxeles: 438836200
34.32user 34.85system 0:21.14elapsed 327%CPU (0avgtext+0avgdata 158568maxresident)k
408inputs+1643624outputs (63major+704439minor)pagefaults 0swaps
cat all.sh >all 
chmod a+x all

real	0m21,219s
user	0m34,381s
sys	0m34,871s
gcc -fopenmp -o main main.c
time ./all.sh
Suma de píxeles: 18903643
Suma de píxeles: 19538173
Suma de píxeles: 508153298
Suma de píxeles: 239735997
Suma de píxeles: 346448124
Suma de píxeles: 156622739
Suma de píxeles: 190237953
Suma de píxeles: 3100604